In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("./pure-methods-ttorrent.csv")
df.shape

(172, 6)

In [3]:
def extract_from_tags(tag, tags):
    search_string = tag + "=[\w]+"
    extracted_tag = re.findall(search_string, tags)[0]
    extracted_tag = re.findall("\=(.*)", extracted_tag)[0]
    if extracted_tag == "true":
        return True
    else:
        return False

In [4]:
cols = ["visibility", "parent-FQN", "method-name", "param-list", "return-type", 
        "local-variables", "conditionals", "multiple-statements", "loops", "parameters", "returns",
        "switches", "ifs", "static", "returns-primitives"]
final_df = pd.DataFrame(columns = cols)
for index, row in df.iterrows():
    final_df.loc[index, 'visibility'] = row['visibility']
    final_df.loc[index, 'parent-FQN'] = row['parent-FQN']
    final_df.loc[index, 'method-name'] = row['method-name']
    final_df.loc[index, 'param-list'] = row['param-list'].strip('[]')
    final_df.loc[index, 'return-type'] = row['return-type']
    final_df.loc[index, 'local-variables'] = extract_from_tags("local_variables", str(row['tags']))
    final_df.loc[index, 'conditionals'] = extract_from_tags("conditionals", str(row['tags']))
    final_df.loc[index, 'multiple-statements'] = extract_from_tags("multiple_statements", str(row['tags']))
    final_df.loc[index, 'loops'] = extract_from_tags("loops", str(row['tags']))
    final_df.loc[index, 'parameters'] = extract_from_tags("parameters", str(row['tags']))
    final_df.loc[index, 'returns'] = extract_from_tags("returns", str(row['tags']))
    final_df.loc[index, 'switches'] = extract_from_tags("switches", str(row['tags']))
    final_df.loc[index, 'ifs'] = extract_from_tags("ifs", str(row['tags']))
    final_df.loc[index, 'static'] = extract_from_tags("static", str(row['tags']))
    final_df.loc[index, 'returns-primitives'] = extract_from_tags("returns_primitives", str(row['tags']))

final_df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,public,com.turn.ttorrent.common.TorrentMetadataImpl,getDirectoryName,,String,False,False,False,False,False,True,False,False,False,False
1,public,com.turn.ttorrent.client.Piece,equals,Object,boolean,False,False,False,False,True,True,False,True,False,True
2,public,com.turn.ttorrent.common.protocol.PeerMessage$...,equals,byte,boolean,False,False,False,False,True,True,False,False,False,True
3,public,com.turn.ttorrent.client.peer.SharingPeer,isInterested,,boolean,False,False,False,False,False,True,False,False,False,True
4,public,com.turn.ttorrent.client.CommunicationManager,getPeersStorage,,PeersStorage,False,False,False,False,False,True,False,False,False,False


### Methods that return objects

In [5]:
(final_df["returns-primitives"] == False).sum()

86

### Methods that accept parameters

In [6]:
(final_df["parameters"] == True).sum()

8

### Methods with if statements

In [7]:
(final_df["ifs"] == True).sum()

1

### Methods wtih conditionals ?:

In [8]:
(final_df["conditionals"] == True).sum()

1

### Methods with local variables

In [9]:
(final_df["local-variables"] == True).sum()

2

### Methods with multiple statements

In [10]:
(final_df["multiple-statements"] == True).sum()

2

In [11]:
instrumentation_candidates_df = pd.DataFrame(columns = cols)
instrumentation_candidates_df = final_df[((final_df['multiple-statements'] == True) | 
                                          (final_df['ifs'] == True) |
                                          (final_df['conditionals'] == True) | 
                                          (final_df['parameters'] == True) | 
                                          (final_df['switches'] == True) |
                                          (final_df['loops'] == True) |
                                          (final_df['local-variables'] == True)) &
                                          (final_df['static'] == False) & 
                                          (final_df['visibility'] == "public")]
print(instrumentation_candidates_df.shape)
instrumentation_candidates_df.to_csv(r'./instrumentation-candidates-pdfbox.csv', index=False)
instrumentation_candidates_df.tail(60)

(7, 15)


,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
1,public,com.turn.ttorrent.client.Piece,equals,Object,boolean,False,False,False,False,True,True,False,True,False,True
2,public,com.turn.ttorrent.common.protocol.PeerMessage$...,equals,byte,boolean,False,False,False,False,True,True,False,False,False,True
62,public,com.turn.ttorrent.common.Optional,orElse,T,T,False,True,False,False,True,True,False,False,False,False
63,public,com.turn.ttorrent.network.AcceptAttachmentImpl,isTimeoutElapsed,long,boolean,False,False,False,False,True,True,False,False,False,True
64,public,com.turn.ttorrent.network.ConnectTask,isTimeoutElapsed,long,boolean,True,False,True,False,True,True,False,False,False,True
65,public,com.turn.ttorrent.network.ReadWriteAttachment,isTimeoutElapsed,long,boolean,True,False,True,False,True,True,False,False,False,True
124,public,com.turn.ttorrent.tracker.TrackerRequestProces...,isBadAddress,String,boolean,False,False,False,False,True,True,False,False,False,True


In [12]:
sorted(instrumentation_candidates_df['parent-FQN'].unique())

['org.apache.fontbox.cff.CFFParser$Format0FDSelect',
 'org.apache.fontbox.cff.CFFParser$Format3FDSelect',
 'org.apache.fontbox.cff.CharStringCommand$Key',
 'org.apache.fontbox.cmap.CIDRange',
 'org.apache.fontbox.cmap.CodespaceRange',
 'org.apache.fontbox.ttf.GlyfSimpleDescript',
 'org.apache.fontbox.ttf.HorizontalMetricsTable',
 'org.apache.fontbox.ttf.KerningSubtable',
 'org.apache.fontbox.ttf.KerningSubtable$PairData0Format0',
 'org.apache.fontbox.ttf.OpenTypeScript$1',
 'org.apache.fontbox.ttf.PostScriptTable',
 'org.apache.fontbox.ttf.VerticalMetricsTable',
 'org.apache.fontbox.type1.Token',
 'org.apache.fontbox.util.BoundingBox',
 'org.apache.pdfbox.cos.COSBoolean',
 'org.apache.pdfbox.debugger.colorpane.DeviceNTableModel',
 'org.apache.pdfbox.debugger.colorpane.IndexedTableModel',
 'org.apache.pdfbox.debugger.ui.PageEntry',
 'org.apache.pdfbox.filter.ASCII85InputStream',
 'org.apache.pdfbox.pdfparser.PDFXRefStream',
 'org.apache.pdfbox.pdfparser.XrefTrailerResolver',
 'org.apach